In [ ]:
#;.pykx.disableJupyter()

In [ ]:
# https://code.kx.com/pykx/3.0/examples/jupyter-integration.html#q-first-mode
import pykx as kx
kx.util.jupyter_qfirst_enable()

 <img src="../qbies.png" style="width: 100px;padding-right:5px;padding-top:1px;padding-left:5px;" align="left"/>

# Scripting And Logging Practical Guidance

## Hiding Source Code

We may wish to hide the contents of our code when sharing scripts with other sources, such as to protect intellectual property. This can be achieved during runtime with the system command [\\_](https://code.kx.com/q/basics/syscmds/#_-hide-q-code) which will prevent the q code from being read or serialized.

A new file will be created on our filesystem with an underscore suffix:

Let's first create another .q text file and name it protected.q. Create a function inside this which prints Hello World -
<code>
sayhello:{0N!"hello world"};
</code>

The `\_` command will protect the script


In [1]:
\_ protected.q 

`protected.q_


Loading non-protected script

In [2]:
\l protected.q

In [3]:
sayhello

{0N!"hello world"}


Loading protected script


In [4]:
\l protected.q_

In [5]:
sayhello

locked


In [6]:
sayhello[]

"hello world"


"hello world"


## Using .Q.def for Command Line Arguments

[.Q.def](https://code.kx.com/q/ref/dotq/#qdef) Provides defaults and types for command line arguments parsed with [.Q.opt](https://code.kx.com/q/ref/dotq/#qopt-command-parameters). This function takes a dictionary of the command line args (as symbols) and default values.

Usage : qdefscript.q 
<code>
d:.Q.def[\`abc\`xyz\`efg!(1;2.;\`a)].Q.opt .z.x;
0N!d;
exit 0
</code>

In [7]:
system["env QHOME=/opt/kx/q q qdefscript.q"] //since no parameters are provided. It'll consider the default values.

`abc`xyz`efg!(1;2f;`a)


In [8]:
system["env QHOME=/opt/kx/q q qdefscript.q -abc 5 -xyz 432 -efg Henry"] // Notice how the strings are casted automatically to the respective types

`abc`xyz`efg!(5;432f;`Henry)


In [9]:
system["env QHOME=/opt/kx/q q qdefscript.q -abc 5 -xyz 432"] //Here it'll consider the default value for efg

`abc`xyz`efg!(5;432f;`a)


## Using Namespaces for Logging

We should always define a namespace .log to logically organize our logging functions. This is an industry-wide best practice. It is also helpful to log the time using [.z.p](https://code.kx.com/q/ref/dotz/#zp-local-timestamp). We can do this as follows-

In [10]:
\d .log
out:{-1 string[.z.p]," ### INFO ### ",x};
err:{-2 string[.z.p]," ### ERROR ### ",x};

\d .
sayhello:{.log.out["Hello ",x]};

sayhello["Nicolas"]

2020.08.06D08:43:09.624438000 ### INFO ### Hello Nicolas


-1


Furthermore, we could also log some of the critical memory statistics provided by [.Q.w[]](https://code.kx.com/q/ref/dotq/#qw-memory-stats)

In [11]:
.Q.w[]

used| 889232
heap| 67108864
peak| 67108864
wmax| 0
mmap| 0
mphy| 8589934592
syms| 1514
symw| 80184


## Script Layout

To help yourself, and others who may read the script you write, it’s good practice to have a common layout of various components of your scripts.

A typical script outline may be broken as follows:

 - Capture command line args
 - Define variables
 - Define functions
 - Define main function
 - Call main to start


This is an example of a well written script

In [ ]:
/command line args
d:first each .Q.def[enlist[`name]!enlist enlist "Joe"] .Q.opt .z.x;
/variable definitions
currentTime:.z.t;
/function definitions
sayhello:{0N!"Hello ",x};
telltime:{0N!"It is currently ",string x};

/main function call
main:{
  sayhello d[`name];
  telltime currentTime;
  exit 0;
 };

/start script
@[main;`;{0N!"Error running main: ",x;exit 1}];

"Hello Joe"
"It is currently 08:46:32.076"
